Julia Zoń, 406998, 
Grupa 3

In [1]:
import psycopg2 as pg
import pandas as pd

connection = pg.connect(host='pgsql-196447.vipserv.org', port=5432, dbname='wbauer_adb', user='wbauer_adb', password='adb2020')

In [2]:
#Zapytanie o liczbę kategorii filmów w wypożyczalni
df = pd.read_sql('select count(distinct name) from category',con=connection)
df

/Users/juliazon/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,count
0,16


Zapytanie o liczbę kategorii za pomocą instrukcji COUNT, która liczy wszystkie wiersze spełniające podane kryterium. Klauzula DISTINCT zduplikowane w wyniku zapytania wiersze.

In [3]:
#Wyświetlenie listy kategorii w kolejności alfabetycznie
df1 = pd.read_sql('select name from category',con=connection)
df1

/Users/juliazon/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,name
0,Action
1,Animation
2,Children
3,Classics
4,Comedy
5,Documentary
6,Drama
7,Family
8,Foreign
9,Games


Najprostsze użycie polecenia SELECT. Wyszukane kategorie są od razu porządkowane alfabetycznie.

In [4]:
#Zapytanie o najmłodszy film w do wypożyczenia
txt =   """
        SELECT release_year, title
        FROM film
        ORDER BY 
            release_year DESC
        FETCH FIRST ROW ONLY;
        """

df2 = pd.read_sql(txt,con=connection)
print(df2)

   release_year            title
0          2006  Chamber Italian


/Users/juliazon/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


W celu znalezienia najmłodszego filmu, użyłam klauzuli ORDER BY by uporządkować filmy. Znacznik ASC oznacza, że filmy są porządkowane według roku wydania - od najmłodszego do nastarszego (lata porządkowane malejąco). Za pomocą klauzuli FETCH wybieram jedynie pierwszy wiersz, czyli najmłodszy film. 

In [5]:
#Zapytanie o najstarszy film w do wypożyczenia
txt =   """
        SELECT 
            release_year, 
            title
        FROM 
            film
        ORDER BY 
            release_year ASC
        FETCH FIRST ROW ONLY;
        """

df3 = pd.read_sql(txt,con=connection)
print(df3)

   release_year            title
0          2006  Chamber Italian


/Users/juliazon/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


Odnalezienie najstarszego filmy wykonujemy analogicznie - z tą różnicą, że porządkujemy filmy od najstarszego do najmłodszego, za co odpowiada znacznik ASC, który porządkuje filmy ze względu na rok wydania rosnąco. Uzyskujemy ten sam wynik, gdyż wszystkie filmy z bazy zostały wydane w tym samym roku.

In [6]:
#Zapytanie ile wypożyczeń odbyło się między 2005-07-01 a 2005-08-01
txt =   """
        SELECT
            count(*)
        FROM
            rental
        WHERE
            rental_date BETWEEN '2005-07-01 00:00:00'
        AND '2005-08-01 23:59:59'
        """
df4 = pd.read_sql(txt,con=connection)
print(df4)           

/Users/juliazon/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


   count
0   7380


In [7]:
#Zapytanie ile wypożyczeń odbyło się między 2010-01-01 a 2011-02-01
txt =   """
        SELECT
            count(*)
        FROM
            rental
        WHERE
            rental_date BETWEEN '2010-01-01 00:00:00'
        AND '2011-02-01 23:59:59'
        """
df4 = pd.read_sql(txt,con=connection)
print(df4) 

/Users/juliazon/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


   count
0      0


Ponownie korzystamy z funkcji COUNT. Dodatkowo, by wyznaczyć ilość wypożyczeń w podanym okresie, używamy klauzuli BETWEEN podając dwie daty.

In [8]:
#Zapytanie o największą płatność wypożyczenia
txt =   """
        SELECT
            amount
        FROM
            payment
        ORDER BY 
            amount DESC 
        FETCH FIRST ROW ONLY;
        """
df5 = pd.read_sql(txt,con=connection)
print(df5) 

/Users/juliazon/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


   amount
0   11.99


Ponownie porządkujemy wiersze za pomocą klauzuli ORDER BY, a następnie wybieramy pierwszy z nich dzięki klauzuli FETCH.

In [9]:
#Zapytanie o wszystkich klientów z Polski, Nigerii i Bangladeszu
txt =   """
        SELECT
            first_name,
            last_name, 
            cntr.country
        FROM
            customer c
        INNER JOIN address a
            ON a.address_id = c.address_id
        INNER JOIN city ct
            ON ct.city_id = a.city_id
        INNER JOIN country cntr
            ON cntr.country_id = ct.country_id
        WHERE
            cntr.country IN ('Poland', 'Nigeria', 'Bangladesh')
        """
df6 = pd.read_sql(txt,con=connection)
print(df6) 

/Users/juliazon/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


   first_name  last_name     country
0       Sonia    Gregory     Nigeria
1       Brian      Wyman      Poland
2      Sidney   Burleson      Poland
3      Rodney    Moeller     Nigeria
4     Stephen     Qualls  Bangladesh
5       Velma      Lucas     Nigeria
6     Marilyn       Ross     Nigeria
7       Elsie     Kelley     Nigeria
8      Gladys   Hamilton     Nigeria
9       Frank   Waggoner  Bangladesh
10   Marjorie     Tucker      Poland
11      Carol     Garcia     Nigeria
12       Leah     Curtis      Poland
13      Ruben      Geary      Poland
14       Olga    Jimenez     Nigeria
15     Bertha   Ferguson     Nigeria
16     Tracey    Barrett     Nigeria
17         Jo     Fowler     Nigeria
18    Johnnie   Chisholm      Poland
19    Wallace      Slone     Nigeria
20   Michelle      Clark  Bangladesh
21    Russell    Brinson      Poland
22     Jimmie  Eggleston      Poland
23  Constance       Reid     Nigeria


W tym przypadku potrzebne informacje do uzyskania podanych danych znajdują się w różnych zbiorach. Do takiego łączenia danych służy klauzula JOIN. Dzięki niej uzyskujemy jednoczesny dostęp do danych powiązanych ze sobą, ale znajdujących się w innych zbiorach.

In [10]:
#Zapytanie o miejsce zamieszkania członków personelu
txt =   """
        SELECT
            first_name,
            last_name,
            ct.city,
            cntr.country
        FROM
            staff s
        INNER JOIN address a
            ON a.address_id = s.address_id
        INNER JOIN city ct
            ON ct.city_id = a.city_id
        INNER JOIN country cntr
            ON cntr.country_id = ct.country_id
        """
df7 = pd.read_sql(txt,con=connection)
print(df7) 

/Users/juliazon/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


  first_name last_name        city    country
0       Mike   Hillyer  Lethbridge     Canada
1        Jon  Stephens   Woodridge  Australia


In [11]:
#Zapytanie ilu pracowników mieszka w Argentynie lub Hiszpanii
txt =   """
        SELECT
            count(*)
        FROM
            staff s
        INNER JOIN address a
            ON a.address_id = s.address_id
        INNER JOIN city ct
            ON ct.city_id = a.city_id
        INNER JOIN country cntr
            ON cntr.country_id = ct.country_id
        WHERE
            cntr.country IN ('Argentina', 'Spain')
        """
df8 = pd.read_sql(txt,con=connection)
print(df8) 

/Users/juliazon/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


   count
0      0


Wykorzystanie klauzuli WHERE, dzięki której można w łatwy sposób sprawdzić, czy wyszukane dane spełniają określone warunki. W tym przypadku sprawdziłam, czy kraj zamieszkania pracownika to Argentyna lub Hiszpania.

In [12]:
#Zapytanie jakie kategorie zostały wypożyczone przez klientów
txt =   """
        SELECT
            DISTINCT name
        FROM
            category c
        INNER JOIN film_category fc
            ON fc.category_id = c.category_id
        INNER JOIN inventory i
            ON i.film_id = fc.film_id
        INNER JOIN rental r
            ON r.inventory_id = i.inventory_id
        """
df9 = pd.read_sql(txt,con=connection)
print(df9) 

/Users/juliazon/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


           name
0        Sports
1      Classics
2           New
3        Family
4        Comedy
5     Animation
6        Travel
7         Music
8         Drama
9        Horror
10       Sci-Fi
11        Games
12  Documentary
13      Foreign
14       Action
15     Children


In [13]:
#Zapytanie o wszystkie kategorie filmów wypożyczonych w Ameryce
txt =   """
        SELECT
            name
        FROM
            category c
        INNER JOIN film_category fc
            ON fc.category_id = c.category_id
        INNER JOIN inventory i
            ON i.film_id = fc.film_id
        INNER JOIN rental r
            ON r.inventory_id = i.inventory_id
        INNER JOIN payment p
            ON p.customer_id = r.customer_id
        INNER JOIN staff s
            ON s.staff_id = p.staff_id
        INNER JOIN store st
            ON st.store_id = s.store_id
        INNER JOIN address a
            ON a.address_id = st.address_id
        INNER JOIN city ct
            ON ct.city_id = a.city_id
        INNER JOIN country cntr
            ON cntr.country_id = ct.country_id
        WHERE
            cntr.country = 'United States'
        """
df10 = pd.read_sql(txt,con=connection)
print(df10) 

/Users/juliazon/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


Empty DataFrame
Columns: [name]
Index: []


In [14]:
txt =   """
        SELECT
            country
        FROM
            store st
        INNER JOIN address a
            ON a.address_id = st.address_id
        INNER JOIN city ct
            ON ct.city_id = a.city_id
        INNER JOIN country cntr
            ON cntr.country_id = ct.country_id
        """
df11 = pd.read_sql(txt,con=connection)
print(df11) 

     country
0     Canada
1  Australia


/Users/juliazon/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


Brak wypożyczeń w Ameryce - sklepy znajdują się jedynie w Kanadzie i Australii.

In [15]:
#Zapytanie o tytuły, w których grali: Olympia Pfeiffer lub Julia Zellweger lub Ellen Presley
txt =   """
        SELECT
            DISTINCT title
        FROM
            category c
        INNER JOIN film_category fc
            ON fc.category_id = c.category_id   
        INNER JOIN film f
            ON f.film_id = fc.film_id
        INNER JOIN film_actor fa
            ON fa.film_id = f.film_id
        INNER JOIN actor a
            ON a.actor_id = fa.actor_id
        WHERE
            (a.first_name = 'Ellen' AND a.last_name = 'Presley') OR
            (a.first_name = 'Julia' AND a.last_name = 'Zellweger') OR
            (a.first_name = 'Olympia' AND a.last_name = 'Pfeiffer')
        """
df12 = pd.read_sql(txt,con=connection)
print(df12) 

/Users/juliazon/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


                   title
0        Hollow Jeopardy
1       Rider Caddyshack
2        Idols Snatchers
3     Magnolia Forrester
4         Image Princess
..                   ...
64     Firehouse Vietnam
65  Streetcar Intentions
66      Effect Gladiator
67          Outlaw Hanky
68          Women Dorado

[69 rows x 1 columns]


Ponowne wykorzystanie klauzuli JOIN i WHERE, które pozwala na łatwy dostęp do danych, pomimo wielu warunków.